In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.special as sp
import os as os
import multiprocessing
from joblib import Parallel, delayed
from time import time as ti
from skimage.restoration import denoise_wavelet
import pickle
#import CoreFunctions as cf
import sys
import random
import psutil
import gc

In [2]:
DataFolder = '/sciclone/scr10/dchendrickson01/Recordings2/'
DataFolder = '/scratch/Recordings2/'

DateString = 'Good'
LastGoodModel = 40

TIME_STEPS = 1200
Skips = 600
RollSize = 50

LastSuccesfull = 40

tic = ti()
start = tic

MemoryProtection = True


In [3]:
%matplotlib inline

In [4]:
FilesPerRun = 5
ConcurrentFiles = 5

In [5]:
def RollingStdDevFaster(RawData, SmoothData, RollSize = 25):

    Diffs = RawData - SmoothData
    del RawData, SmoothData
    
    Sqs = Diffs * Diffs
    del Diffs
    
    Sqs = Sqs.tolist() 
    Sqs.extend(np.zeros(RollSize))
    mSqs = np.matrix(Sqs)
    
    for i in range(RollSize):
        Sqs.insert(0, Sqs.pop())
        mSqs = np.concatenate((np.matrix(Sqs),mSqs))
    
    sVect = mSqs.sum(axis=0)
    eVect = (mSqs!=0).sum(axis=0)
    del mSqs, Sqs
    
    VarVect = sVect / eVect
    StdDevs = np.sqrt(VarVect)
    return np.asarray(StdDevs[:-RollSize].T)

def SquelchPattern(DataSet, StallRange = 5000, SquelchLevel = 0.02, verbose = False):
    
    SquelchSignal = np.ones(len(DataSet))
    if verbose:
        print(len(SquelchSignal))
        
    for i in range(len(DataSet)-2*StallRange):
        if np.average(DataSet[i:i+StallRange]) < SquelchLevel:
            SquelchSignal[i+StallRange]=0

    return SquelchSignal

def split_list_by_zeros(original_list, ones_list):
    # modified split_list_by_ones function to instead split by the zeros.
    #
    #
    # Created with Bing AI support
    #  1st request: "python split list into chunks based on value"
    #  2nd request: "I want to split the list based on the values in a second list.  Second list is all 1s and 0s.  I want all 0s removed, and each set of consequtive ones as its own item"
    #  3rd request: "That is close.  Here is an example of the two lists, and what I would want returned: original_list = [1, 2, 3, 8, 7, 4, 5, 6, 4, 7, 8, 9]
    #                ones_list =     [1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1]
    #                return: [[1, 2, 3, 8], [4, 5, 6], [8,9]]"
    #
    #This is the function that was created and seems to work on the short lists, going to use for long lists
    
    result_sublists = []
    sublist = []

    for val, is_one in zip(original_list, ones_list):
        if not is_one:
            sublist.append(val)
        elif sublist:
            result_sublists.append(sublist)
            sublist = []

    # Add the last sublist (if any)
    if sublist:
        result_sublists.append(sublist)

    return result_sublists

# Generated training sequences for use in the model.
def create_sequences(values, time_steps=TIME_STEPS, skips = Skips):
    output = []
    for i in range(int((len(values) - time_steps + skips)/skips)):
        output.append(values[i*skips : (i*skips + time_steps)])
    return np.stack(output)

In [6]:
def runFile(file, verbose = False, small = False, index=0, start=ti()):
    noise = verbose
    if file[-4:] == '.csv':    
        dataset = pd.read_csv(DataFolder+file, delimiter =",", header=None, engine='python',on_bad_lines='skip')
        if noise:
            print("File Read", ti()-start)
        dataset = dataset.rename(columns={0:"Day"})
        dataset = dataset.rename(columns={1:"Second"})
        dataset = dataset.rename(columns={2:"FracSec"})
        dataset = dataset.rename(columns={3:"p"})
        dataset = dataset.rename(columns={4:"h"})
        dataset = dataset.rename(columns={5:"v"})
        dataset = dataset.rename(columns={6:"Sensor"})

        #dataset['Second'].replace('',0)
        #dataset['FracSec'].replace('',0)
        #dataset.replace([np.nan, np.inf, -np.inf],0,inplace=True)
        
        #dataset[['Day','Second']] = dataset[['Day','Second']].apply(lambda x: x.astype(int).astype(str).str.zfill(6))
        #dataset[['FracSec']] = dataset[['FracSec']].apply(lambda x: x.astype(int).astype(str).str.zfill(4))

        #dataset["timestamp"] = pd.to_datetime(dataset.Day+dataset.Second+dataset.FracSec,format='%y%m%d%H%M%S%f')
        #dataset["timestamps"] = dataset["timestamp"]

        dataset["p"] = dataset.p - np.average(dataset.p)
        dataset["h"] = dataset.h - np.average(dataset.h)
        dataset["v"] = dataset.v - np.average(dataset.v)
        #dataset["r"] = np.sqrt(dataset.p**2 + dataset.h**2 + dataset.v**2)

        #dataset.index = dataset.timestamp

        dataset["SmoothP"] = denoise_wavelet(dataset.p, method='VisuShrink', mode='soft', wavelet_levels=3, wavelet='sym2', rescale_sigma='True')
        dataset["SmoothH"] = denoise_wavelet(dataset.h, method='VisuShrink', mode='soft', wavelet_levels=3, wavelet='sym2', rescale_sigma='True')
        dataset["SmoothV"] = denoise_wavelet(dataset.v, method='VisuShrink', mode='soft', wavelet_levels=3, wavelet='sym2', rescale_sigma='True')
        #dataset["SmoothR"] = denoise_wavelet(dataset.r, method='VisuShrink', mode='soft', wavelet_levels=3, wavelet='sym2', rescale_sigma='True')

        if noise:
            print("Data Cleaned", ti()-start, len(dataset.p))

        '''
        RawData = dataset.v
        SmoothData = dataset.SmoothV
        RollSize = 25

        Diffs = RawData - SmoothData

        Sqs = Diffs * Diffs

        Sqs = Sqs.tolist() 

        Sqs.extend(np.zeros(RollSize))

        mSqs = np.matrix(Sqs)

        for i in range(RollSize):
            Sqs.insert(0, Sqs.pop())
            mSqs = np.concatenate((np.matrix(Sqs),mSqs))

        sVect = mSqs.sum(axis=0)
        eVect = (mSqs!=0).sum(axis=0)

        VarVect = sVect / eVect

        StdDevs = np.sqrt(VarVect)

        StdDevsZ = np.asarray(StdDevs)

        StdDevsZ=np.append(StdDevsZ,[0])

        StdDevsZ = np.asarray(StdDevsZ.T[:len(dataset.p)])

        if noise:
            print("Size StdDevsZ", ti()-start, np.shape(StdDevsZ))

        #StdDevsZ = np.nan_to_num(StdDevsZ)

        #StdDevsZ[StdDevsZ == np.inf] = 0
        #StdDevsZ[StdDevsZ == -np.inf] = 0

        if noise:
            print("cleaned", ti()-start, np.shape(StdDevsZ))

        SmoothDevZ = denoise_wavelet(StdDevsZ, method='VisuShrink', mode='soft', wavelet='sym2', rescale_sigma='True')

        if noise:
            print("denoise 1", ti()-start, np.shape(StdDevsZ))

        #SmoothDevZa = cf.Smoothing(StdDevsZ, 3, wvt='sym2', dets_to_remove=2, levels=3)
        #SmoothDevZ = np.ravel(SmoothDevZ[0,:])

        #SmoothDevZ = SmoothDevZ.tolist()

        if noise:
            print("denoise 2", ti()-start, np.shape(SmoothDevZ))

        #ataset["SmoothDevZ"] = SmoothDevZ

        SmoothDevZ[np.isnan(SmoothDevZ)]=0
        
        Max = np.max(SmoothDevZ)

        
        
        if noise:
            print("Max", ti()-start, np.shape(Max), Max)

        buckets = int(Max / 0.005) + 1
        bins = np.linspace(0,buckets*0.005,buckets+1)
        counts, bins = np.histogram(SmoothDevZ,bins=bins)

        CummCount = 0
        HalfWay = 0
        for i in range(len(counts)):
            CummCount += counts[i]
            if CummCount / len(SmoothDevZ) >= 0.5:
                if HalfWay == 0:
                    HalfWay = i

        SquelchLevel = bins[HalfWay] 
        if noise:
            print("SmoothDevz size", np.shape(SmoothDevZ))
    
        dataset["IsMoving"] = SquelchPattern(SmoothDevZ, 2000, SquelchLevel, verbose=noise)
        '''
        if noise:
            print("Squelch Made", ti()-start)
        #dataset["velocity"] = getVelocity(dataset.p, dataset.FracSec, dataset.IsMoving, 2)
        #if noise:
        #    print("Velocity Calculated.  File done: ",file)

        #df_pr = split_list_by_ones(dataset.p, dataset.IsMoving)
        #df_hr = split_list_by_ones(dataset.h, dataset.IsMoving)
        #df_vr = split_list_by_ones(dataset.v, dataset.IsMoving)
        #df_rrr = split_list_by_ones(dataset.r, dataset.IsMoving)
        #df_ps = split_list_by_zeros(dataset.SmoothP, dataset.IsMoving)
        #df_hs = split_list_by_zeros(dataset.SmoothH, dataset.IsMoving)
        #df_vs = split_list_by_zeros(dataset.SmoothV, dataset.IsMoving)
        df_ps = create_sequences(dataset.SmoothP)
        df_hs = create_sequences(dataset.SmoothH)
        df_vs = create_sequences(dataset.SmoothV)
        #df_rs = split_list_by_zeros(dataset.SmoothR, dataset.IsMoving)

        del dataset
        
        SmoothMove = []
        for i in range(len(df_ps)):
            SmoothMove.append(np.vstack((df_ps[i],df_hs[i],df_vs[i]))) #,df_rs[i])))
        SmoothStop = []
        for i in range(len(df_pr)):
            SmoothStop.append(np.vstack((df_pr[i],df_hr[i],df_vr[i]))) #,df_rs[i])))
        
        if verbose:
            print("Split by ones", ti()-start)


        '''df_p=[0]
        df_h=[0]
        df_v=[0]
        df_r=[0]
        df_rp=[0]
        df_rh=[0]
        df_rv=[0]
        df_rr=[0]
        for i in range(len(df_ps)):
            df_p += df_ps[i]
            df_h += df_hs[i]
            df_v += df_vs[i]
            df_r += df_rs[i]
            df_rp += df_pr[i]
            df_rh += df_hr[i]
            df_rv += df_vr[i]
            df_rr += df_rrr[i]
        '''
        if verbose:
            print('format changed', ti()-start, len(MatsSmooth))

        return SmoothMove,SmoothStop
    else:
        return ['fail','fail']
        
        #if verbose:
        #    print('Data normalized', ti()-start)

        #return df_p, df_h, df_v, df_r, df_rp, df_rh, df_rv, df_rr


In [7]:
import CoreFunctions as cf

In [8]:
def runWrapper(file_path, verbose=True, small=False, index=0, start=ti()):
    try:
        rtrn = runFile(file_path, verbose, small, index, start)
        return rtrn
    except Exception as e:
        with open('BadInputs.text', 'a') as bad_file:
            bad_file.write(file_path + '\n')
        return np.zeros((10, 10, 3))

In [9]:
def PrintWrap(Mat):
    localPrints = []
    lenm = np.shape(Mat)[1]
    slices = int(lenm/TIME_STEPS)
    for i in range(slices):
        temp = (cf.makeMPFast(Mat[:3,i*TIME_STEPS:(i+1)*TIME_STEPS], wvt = 'sym4', scales = 32, spacer = 2, title = ''))
        localPrints.append(temp.astype(np.float32)/255.0)
    return localPrints

# Start Machine Learning
## Using Autoencoder with Kears and Tensorflow
cite: https://pyimagesearch.com/2020/02/17/autoencoders-with-keras-tensorflow-and-deep-learning/

In [10]:
# import the necessary packages
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Conv2DTranspose
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Reshape
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
import numpy as np

2024-12-06 14:27:22.565063: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-06 14:27:22.587013: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-06 14:27:22.593736: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-06 14:27:22.609837: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [11]:
class ConvAutoencoder:
    @staticmethod
    def build(width, height, depth, filters=(32, 64), latentDim=24):
        inputShape = (height, width, depth)
        chanDim = -1
        inputs = Input(shape=inputShape)
        x = inputs

        for f in filters:
            x = Conv2D(f, (3, 3), strides=2, padding="same")(x)
            x = LeakyReLU(alpha=0.2)(x)
            x = BatchNormalization(axis=chanDim)(x)

        volumeSize = K.int_shape(x)
        print("Volume Size:", volumeSize)
        x = Flatten()(x)
        latent = Dense(latentDim)(x)

        encoder = Model(inputs, latent, name="encoder")

        latentInputs = Input(shape=(latentDim,))
        flattenedVolumeSize = int(np.prod(volumeSize[1:]))
        print("Flattened Volume Size:", flattenedVolumeSize)
        x = Dense(flattenedVolumeSize)(latentInputs)
        x = Reshape((volumeSize[1], volumeSize[2], volumeSize[3]))(x)

        for f in filters[::-1]:
            x = Conv2DTranspose(f, (3, 3), strides=2, padding="same")(x)
            x = LeakyReLU(alpha=0.2)(x)
            x = BatchNormalization(axis=chanDim)(x)

        x = Conv2DTranspose(depth, (3, 3), padding="same")(x)
        outputs = Activation("sigmoid")(x)

        decoder = Model(latentInputs, outputs, name="decoder")
        autoencoder = Model(inputs, decoder(encoder(inputs)), name="autoencoder")

        return (encoder, decoder, autoencoder)


In [12]:
# set the matplotlib backend so figures can be saved in the background
#import matplotlib
#matplotlib.use("Agg")
# import the necessary packages
#from pyimagesearch.convautoencoder import ConvAutoencoder
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.datasets import mnist
import matplotlib.pyplot as plt
import numpy as np
import argparse
import cv2
# construct the argument parse and parse the arguments

In [13]:
import random
import tensorflow as tf
from tensorflow.keras.models import load_model

In [14]:
from tensorflow.keras.callbacks import Callback

In [15]:
import keras
import re

In [16]:
directory = '/scratch/models/'
target_file = f'3DFP_{DateString}_{str(LastGoodModel).zfill(3)}_good2_ae.keras'

ImageShape=[5,32,600,3]

encoder = load_model(directory+target_file[:-8]+'en.keras')
decoder = load_model(directory+target_file[:-8]+'de.keras')

autoencoder_input = Input(shape=(ImageShape[1], ImageShape[2], ImageShape[3]))

# Pass the input through the encoder and decoder
encoded_repr = encoder(autoencoder_input)
reconstructed = decoder(encoded_repr)

# Create the reassembled autoencoder model
autoencoder = Model(autoencoder_input, reconstructed)

2024-12-06 14:27:25.959439: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: UNKNOWN ERROR (34)


In [17]:
print(encoder.summary())
print(decoder.summary())
print(autoencoder.summary())
#if LastSuccesfull != 0:
#    print(reautoencoder.summary())

Model: "encoder"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 32, 600, 3)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 16, 300, 64)    │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu (LeakyReLU)         │ (None, 16, 300, 64)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 16, 300, 64)    │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 8, 150, 32)     │        18,464 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_1 (LeakyReLU)       │ (None, 8, 150, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 8, 150, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 38400)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 32)             │     1,228,832 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,249,472 (4.77 MB)

 Trainable params: 1,249,280 (4.77 MB)

 Non-trainable params: 192 (768.00 B)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 32, 600, 3)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ encoder (Functional)            │ (None, 32)             │     1,249,472 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ decoder (Functional)            │ (None, 32, 600, 3)     │     1,297,059 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,546,531 (9.71 MB)

 Trainable params: 2,546,147 (9.71 MB)

 Non-trainable params: 384 (1.50 KB)

None


In [18]:
def getTensors(file_list):

    Mats=[]

    AllDatas = Parallel(n_jobs=ConcurrentFiles)(delayed(runWrapper)(file_list[i], False, False, 0, ti()) for i in range(FilesPerRun))
    
    for fileResponse in AllDatas:
        for Mat in fileResponse:
            Mats.append(Mat)
            
    if MemoryProtection:
        del AllDatas
        print('RAM after AllData:', psutil.virtual_memory()[2], len(Mats))        
    lengths = []
    rejects = []
    Keeps = []
    
    for Mat in Mats:
        spm = np.shape(Mat)
        if len(spm) > 1:
            lenM = spm[1]
        else:
            lenM = 1
        if (lenM > 1250):
            lengths.append(lenM)
            Keeps.append(Mat)
        else:
            rejects.append(lenM)
    
    if MemoryProtection:
        del Mats
    
    Prints = []
    
    
    AllPrints = Parallel(n_jobs=8)(delayed(PrintWrap)(Mat) for Mat in Keeps)
    
    if MemoryProtection:
        del Keeps
        print('RAM after Keeps:', psutil.virtual_memory()[2])
    for group in AllPrints:
        for fprint in group:
            Prints.append(fprint[:, ::2, :])
    
    if MemoryProtection:
        del AllPrints
    random.shuffle(Prints)
    
    for i, image in enumerate(Prints):
        if not isinstance(image, np.ndarray):
            Prints[i] = np.array(image, dtype=np.float32)
        elif image.dtype != np.float32:
            Prints[i] = image.astype(np.float32)
    
    # Stack the images into a single NumPy array
    prints_array = np.stack(Prints, axis=0)
    
    if MemoryProtection:
        del Prints
        print('RAM after Prints:', psutil.virtual_memory()[2])
    # Convert the NumPy array to a TensorFlow tensor
    trX = tf.convert_to_tensor(prints_array)
    
    if MemoryProtection:
        del prints_array
    return trX

In [19]:
files=['230704 recording4.csv',
       '230421 recording1.csv',
       '230420 recording2.csv',
       ]

In [20]:
Mats=[]

In [21]:
AllDatas = []
AllDatas.append('Temp')

In [22]:
for file in files:
    AllDatas.append(runWrapper(file, False, False, 0, ti()) )

In [23]:
AllDatas = AllDatas[1:]

In [24]:
len(AllDatas)

3

In [25]:
for fileResponse in AllDatas:
    for Mat in fileResponse:
        Mats.append(Mat)


In [26]:
for i in range(len(Mats)):
    if np.shape(Mats[i]) != (3,1200):
        print(i, np.shape(Mats[i]), Mats[i])

0 (10, 3) [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
1 (10, 3) [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
2 (10, 3) [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
3 (10, 3) [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
4 (10, 3) [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
5 (10, 3) [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
6 (10, 3) [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
7 (10, 3) [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 

In [27]:
if MemoryProtection:
    del AllDatas
    print('RAM after AllData:', psutil.virtual_memory()[2], len(Mats))        

RAM after AllData: 6.1 30


In [28]:
lengths = []
rejects = []
Keeps = []

In [29]:
for Mat in Mats:
    spm = np.shape(Mat)
    if len(spm) > 1:
        lenM = spm[1]
    else:
        lenM = 1
    if (lenM >= 1200):
        lengths.append(lenM)
        Keeps.append(Mat)
    else:
        rejects.append(lenM)

In [30]:
if MemoryProtection:
    del Mats

In [31]:
Prints = []

In [32]:
AllPrints = Parallel(n_jobs=8)(delayed(PrintWrap)(Mat) for Mat in Keeps)

In [33]:
np.shape(AllPrints)

(0,)

In [34]:
if MemoryProtection:
    del Keeps, rejects, lengths
    print('RAM after Keeps:', psutil.virtual_memory()[2]) 

RAM after Keeps: 6.1


In [35]:
for group in AllPrints:
    for fprint in group:
        Prints.append(fprint[:, ::2, :])

In [36]:
random.shuffle(Prints)

In [37]:
if MemoryProtection:
    del AllPrints
    print('RAM after AllPrints:', psutil.virtual_memory()[2])    

RAM after AllPrints: 6.1


In [38]:
np.shape(Prints)

(0,)

In [39]:
for i, image in enumerate(Prints):
    if not isinstance(image, np.ndarray):
        Prints[i] = np.array(image, dtype=np.float32)
    elif image.dtype != np.float32:
        Prints[i] = image.astype(np.float32)

# Stack the images into a single NumPy array
prints_array = np.stack(Prints, axis=0)

ValueError: need at least one array to stack

In [ ]:
if MemoryProtection:
    del Prints
print('RAM after Prints:', psutil.virtual_memory()[2])
# Convert the NumPy array to a TensorFlow tensor

In [ ]:
trX = tf.convert_to_tensor(prints_array)

In [ ]:
if MemoryProtection:
    del prints_array
print('RAM after Tensor:', psutil.virtual_memory()[2])

In [ ]:
Predictions = autoencoder.predict(trX)

In [ ]:
Deltas = trX-Predictions

print('RAM after Predictions and Deltas:', psutil.virtual_memory()[2])

In [ ]:
np.shape(Deltas)

In [ ]:
ResultsS = []
ResultsA = []
ResultsM = []
ResultsAM = []
for result in Deltas:
    ResultsS.append(np.sum(result))
    ResultsA.append(np.sum(np.abs(result)))
    ResultsM.append(np.mean(result))
    ResultsAM.append(np.mean(np.abs(result)))
    
    
    


In [ ]:
def filter_data(data, min = -np.inf, max = np.inf):
    data = np.array(data)
    data[np.isnan(data)] = 0
    data[np.isinf(data)] = 0
    mean = np.mean(data) 
    std_dev = np.std(data) 
    filtered_data = data[np.abs(data - mean) / std_dev < 2]
    filtered_data = filtered_data[filtered_data > min]
    filtered_data = filtered_data[filtered_data < max]

    return filtered_data

In [ ]:
plt.hist(filter_data(ResultsS), bins=20)
plt.xlabel("Sum Difs")
plt.ylabel("No of samples")
plt.show()
plt.hist((ResultsS), bins=20)
plt.xlabel("Sum Difs")
plt.ylabel("No of samples")
plt.show()

In [ ]:
plt.hist(filter_data(ResultsA,0,50000), bins=20)
plt.xlabel("Sum Abs Value Diffs")
plt.ylabel("No of samples")
plt.show()
plt.hist(ResultsA, bins=20)
plt.xlabel("Sum Abs Value Diffs")
plt.ylabel("No of samples")
plt.show()

In [ ]:
plt.hist(filter_data(ResultsM,-5,2), bins=20)
plt.xlabel("Mean Difs")
plt.ylabel("No of samples")
plt.show()
plt.hist(ResultsM, bins=20)
plt.xlabel("Mean Difs")
plt.ylabel("No of samples")
plt.show()


In [ ]:
plt.hist(filter_data(ResultsAM,0,2), bins=20)
plt.xlabel("Mean Abs Val Difs")
plt.ylabel("No of samples")
plt.show()
plt.hist(ResultsAM, bins=20)
plt.xlabel("Sum Difs")
plt.ylabel("Mean Abs Val Difs")
plt.show()


 - sum difference by color by pixel by time step
 - sum difference by pixel by time step
 - averages diffferences by both
 - graphs of differences over time for a sample

In [ ]:
TimeDiffs = np.sum(Deltas, axis=1)
np.shape(TimeDiffs)

In [ ]:
TimeDiffsPixel = np.sum(TimeDiffs,axis=2)
np.shape(TimeDiffsPixel)

In [ ]:
fig = plt.figure()
plt.plot(TimeDiffs[99,:,0])
plt.plot(TimeDiffs[99,:,1])
plt.plot(TimeDiffs[99,:,2])
plt.plot(TimeDiffsPixel[99])
plt.show()

In [ ]:
Small = []
Large = []

for i in range(len(ResultsAM)):
    if ResultsAM[i]<0.3 and ResultsAM[i]>0.1:
        Small.append(i)
    elif ResultsAM[i]>0.4:
        Large.append(i)

In [ ]:
random.shuffle(Small)
random.shuffle(Large)

In [ ]:
for i in range(5):
    fig = plt.figure()
    plt.plot(TimeDiffs[Small[i],:,0])
    plt.plot(TimeDiffs[Small[i],:,1])
    plt.plot(TimeDiffs[Small[i],:,2])
    plt.plot(TimeDiffsPixel[Small[i]],color='r')
    plt.show()

In [ ]:
for i in range(5):
    fig = plt.figure()
    plt.plot(TimeDiffs[Large[i],:,0])
    plt.plot(TimeDiffs[Large[i],:,1])
    plt.plot(TimeDiffs[Large[i],:,2])
    plt.plot(TimeDiffsPixel[Large[i]],color='r')
    plt.show()

In [ ]:
#Deltas trX Predictions

for j in range(6):
    i = Large[j]
    fig, (ax0, ax1, ax3) = plt.subplots(1,3,figsize=(9,2), dpi=300 )
    ax0.imshow(trX[i], origin='lower',aspect='auto')
    ax0.axis("off")
    ax1.imshow(Predictions[i], origin='lower',aspect='auto')
    ax1.axis("off")
    #ax2.imshow(Deltas[i], origin='lower',aspect='auto')
    #ax2.axis("off")
    ax3.plot(TimeDiffs[i,:,0])
    ax3.plot(TimeDiffs[i,:,1])
    ax3.plot(TimeDiffs[i,:,2])
    #ax3.plot(TimeDiffsPixel[i],color='r')
    
    plt.show()

In [ ]:
#Deltas trX Predictions

for j in range(6):
    i = Small[j]
    fig, (ax0, ax1, ax3) = plt.subplots(1,3,figsize=(9,2), dpi=300 )
    ax0.imshow(trX[i], origin='lower',aspect='auto')
    ax0.axis("off")
    ax1.imshow(Predictions[i], origin='lower',aspect='auto')
    ax1.axis("off")
    #ax2.imshow(Deltas[i], origin='lower',aspect='auto')
    #ax2.axis("off")
    ax3.plot(TimeDiffs[i,:,0])
    ax3.plot(TimeDiffs[i,:,1])
    ax3.plot(TimeDiffs[i,:,2])
    #ax3.plot(TimeDiffsPixel[i],color='r')
    
    plt.show()

In [ ]:
Errors = []

for i in range(len(ResultsAM)):
    if ResultsAM[i]<0.1:
        Errors.append(i)


In [ ]:
random.shuffle(Errors)

In [ ]:
for j in range(6):
    i = Errors[j]
    fig, (ax0, ax1, ax3) = plt.subplots(1,3,figsize=(9,2), dpi=300 )
    ax0.imshow(trX[i], origin='lower',aspect='auto')
    ax0.axis("off")
    ax1.imshow(Predictions[i], origin='lower',aspect='auto')
    ax1.axis("off")
    ax3.plot(TimeDiffs[i,:,0])
    ax3.plot(TimeDiffs[i,:,1])
    ax3.plot(TimeDiffs[i,:,2])
    plt.show()

In [ ]:
for j in range(6):
    i = Errors[j]
    print(np.sum(trX[i]),np.sum(Predictions[i]))